# 1. 匯入常用套件 & 設定資料路徑

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # 另一個繪圖-樣式套件

import warnings
warnings.filterwarnings('ignore')

#系統操作、解壓縮檔案
import os
from zipfile import ZipFile

# 設定 
dir_data = './data/'
dir_submit = './submit/'
filename = 'house-prices.zip'

# 2. 定義Function

In [35]:
# 查看資料內容
def columns_type(df_data):
    dtype_df = df_data.dtypes.reset_index()
    dtype_df.columns = ['Columns', 'Type']
    dtype_df_count = dtype_df.groupby(['Type'])['Columns'].count().reset_index()
    for t, n in zip(dtype_df_count.Type, dtype_df_count.Columns):
        print('類型 : ',t,'\n')
        print('  個數 : ',n,'\n')
        print('  ',list(dtype_df[dtype_df.Type==t].Columns),'\n')
        print('-----------------------------------------------\n')
        
def object_check(df_data):    
    object_type = df_data.select_dtypes(include=["object"]).apply(pd.Series.nunique, axis = 0).reset_index() 
    object_type.columns = ["Column", "Unique Count"]
    display(object_type.head(10))        
        
# 檢查 DataFrame 空缺值的狀態
def na_check(df_data,df_test):
    df_data_na = (df_data.isnull().sum() / len(df_data)) * 100
    df_data_na = df_data_na.drop(df_data_na[df_data_na == 0].index).sort_values(ascending=False)
    train_missing_data = pd.DataFrame({'Train_Missing Ratio' :df_data_na}).reset_index().rename(columns={'index':'Columns'})
    df_test_na = (df_test.isnull().sum() / len(df_test)) * 100
    df_test_na = df_test_na.drop(df_test_na[df_test_na == 0].index).sort_values(ascending=False)
    test_missing_data = pd.DataFrame({'Test_Missing Ratio' :df_test_na}).reset_index().rename(columns={'index':'Columns'})
    missing_data = pd.merge(train_missing_data,test_missing_data, on= 'Columns',how='outer')    
    display(missing_data)

# 3. 匯入資料

In [36]:
# 查看壓縮檔內的資料
f_path = os.path.join(dir_data, filename)
print('Path of read in data: %s' % (f_path))

myzip=ZipFile(f_path)
for i in myzip.filelist:
    print(i.filename)

Path of read in data: ./data/house-prices.zip
data_description.txt
sample_submission.csv
test.csv
train.csv


In [37]:
# 讀取壓縮檔內的資料
f=myzip.open('train.csv')
train=pd.read_csv(f)
f.close()

f=myzip.open('test.csv')
test=pd.read_csv(f)
f.close()

myzip.close()

# 4. ETL & EDA

In [41]:
print('Train data :{}'.format(train.shape))
print('Test data :{}'.format(test.shape))
print('\n')
columns_type(train)
object_check(train)
print('\n')
print('train data na_check')
na_check(train,test)
# print('\n')
# print('test data na_check')
# na_check(test)

Train data :(1460, 81)
Test data :(1459, 80)


類型 :  int64 

  個數 :  35 

   ['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'] 

-----------------------------------------------

類型 :  float64 

  個數 :  3 

   ['LotFrontage', 'MasVnrArea', 'GarageYrBlt'] 

-----------------------------------------------

類型 :  object 

  個數 :  43 

   ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', '

,Column,Unique Count
0,MSZoning,5
1,Street,2
2,Alley,2
3,LotShape,4
4,LandContour,4
5,Utilities,2
6,LotConfig,5
7,LandSlope,3
8,Neighborhood,25
9,Condition1,9




train data na_check


,Columns,Train_Missing Ratio,Test_Missing Ratio
0,PoolQC,99.520548,99.794380
1,MiscFeature,96.301370,96.504455
2,Alley,93.767123,92.666210
3,Fence,80.753425,80.123372
4,FireplaceQu,47.260274,50.034270
5,LotFrontage,17.739726,15.558602
6,GarageYrBlt,5.547945,5.346127
7,GarageType,5.547945,5.209047
8,GarageFinish,5.547945,5.346127
9,GarageQual,5.547945,5.346127


### 訓練資料中，有35個整數型、3個浮點數型和43個類別型；再Missing Values部分，Test資料在較多欄位中有NA
### 分別取出 數量行變數 與 類別行變數

In [46]:
number_features=[]
object_features = []
for dtype, feature in zip(train.dtypes, train.columns):
    if dtype == 'object':
        object_features.append(feature)
    else :
        number_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')
print(f'{len(number_features)} Number_Features : {number_features}\n')

43 Object Features : ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

38 Number_Features : ['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', '

In [48]:
for c1 in object_features:
    train[c1] = train[c1].fillna('None')
for c2 in number_features:
    train[c2] = train[c2].fillna(train[c2].median)

In [51]:
train.isnull().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64

In [ ]:
#X_test = X_test[X_train.columns]
miss_columns = set(X_train.columns) - set(X_test.columns)
 
#add missing dummy columns
for col in miss_columns:
    X_test[col] = 0
    
adu_columns = set(X_test.columns) - set(X_train.columns)
#delete adundant columns:
X_test.drop(list(adu_columns),axis=1,inplace=True)

————————————————
版权声明：本文为CSDN博主「momottyy」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/maotianyi941005/article/details/88582464